# iQuHack 2026 - Quantum Entanglement Distillation Game

In this game, you design quantum circuits to distill noisy Bell pairs and claim edges in a network.

**Game Flow**: Register -> Select Starting Node -> Design Circuits -> Claim Edges -> Score Points

In [4]:
from client import GameClient
from visualization import GraphTool
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import json
from pathlib import Path

## Session Management

Save/load your session to avoid re-registering.

In [5]:
SESSION_FILE = Path("session.json")

def save_session(client):
    if client.api_token:
        with open(SESSION_FILE, "w") as f:
            json.dump({"api_token": client.api_token, "player_id": client.player_id, "name": client.name}, f)
        print(f"Session saved.")

def load_session():
    if not SESSION_FILE.exists():
        return None
    with open(SESSION_FILE) as f:
        data = json.load(f)
    client = GameClient(api_token=data.get("api_token"))
    client.player_id = data.get("player_id")
    client.name = data.get("name")
    status = client.get_status()
    if status:
        print(f"Resumed: {client.player_id} | Score: {status.get('score', 0)} | Budget: {status.get('budget', 0)}")
        return client
    
    return None

In [6]:
# Try to resume existing session
client = load_session()

if not client:
    print("No saved session. Register below.")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Step 1: Register

Skip this if you resumed a session above.

In [ ]:
if client and client.api_token:
    print(f"Already registered as {client.player_id}")
    status = client.get_status()
    candidates = status.get("starting_candidates", [])
    if candidates:
        print(f"\nStarting candidates ({len(candidates)}):")
        for c in candidates:
            print(f"  - {c['node_id']}: {c['utility_qubits']} qubits, +{c['bonus_bell_pairs']} bonus")
else:
    client = GameClient()
    
    # CHANGE THESE to your unique values
    PLAYER_ID = "testerman"
    PLAYER_NAME = "Flip Flipplesin"
    
    result = client.register(PLAYER_ID, PLAYER_NAME, location="remote")
    
    if result.get("ok"):
        print(f"Registered! Token: {client.api_token[:20]}...")
        candidates = result["data"].get("starting_candidates", [])
        print(f"\nStarting candidates ({len(candidates)}):")
        for c in candidates:
            print(f"  - {c['node_id']}: {c['utility_qubits']} qubits, +{c['bonus_bell_pairs']} bonus")
        save_session(client)
    else:
        print(f"Failed: {result.get('error', {}).get('message')}")

Already registered as diyamagnetism


In [596]:
print(status)

{'player_id': 'diyamagnetism', 'name': 'Diya Naik', 'score': 16, 'budget': 70, 'starting_node': 'Brussels, Belgium', 'is_active': True, 'owned_nodes': ['Brussels, Belgium', 'Paris, France', 'Leuven, Belgium', 'Ghent, Belgium', 'Antwerp, Belgium', 'Lille, France'], 'owned_edges': [['Brussels, Belgium', 'Paris, France'], ['Brussels, Belgium', 'Leuven, Belgium'], ['Brussels, Belgium', 'Leuven, Belgium'], ['Brussels, Belgium', 'Leuven, Belgium'], ['Brussels, Belgium', 'Leuven, Belgium'], ['Brussels, Belgium', 'Lille, France'], ['Ghent, Belgium', 'Paris, France'], ['Antwerp, Belgium', 'Brussels, Belgium']]}


## Step 2: Select Starting Node

In [120]:
status = client.get_status()



if status.get('starting_node'):
    print(f"Starting node: {status['starting_node']}")
    print(f"Budget: {status['budget']} | Score: {status['score']}")
else:
    print("Select a starting node from the candidates shown above.")
    # Uncomment and modify:
    result = client.select_starting_node("Toulouse, France")
    print(result)

Select a starting node from the candidates shown above.
{'ok': True, 'data': {'success': True, 'starting_node': 'Toulouse, France', 'score': 0, 'budget': 75}}


## Step 3: Explore the Network

In [828]:
client.print_status()

Player: diyamagnetism (Diya Naik)
Score: 129 | Budget: -1 bell pairs
Active: No
Starting node: Brussels, Belgium
Owned: 57 nodes, 82 edges
Claimable edges: 110
  - ['Charlottesville, VA', 'College Park, MD']: threshold=0.90, difficulty=2
  - ['Albany, NY', 'Rome, NY']: threshold=0.90, difficulty=1
  - ['Miami, FL', 'Nassau, Bahamas']: threshold=0.90, difficulty=2
  ... and 107 more


## Step 4: Design a Distillation Circuit

For N Bell pairs, qubits are paired outside-in:
- Pair 0: qubits {0, 2N-1}
- Pair 1: qubits {1, 2N-2}
- ...
- Final pair: qubits {N-1, N}

Design a circuit that improves fidelity through LOCC operations.

In [816]:
def create_distillation_circuit():
    # 2-to-1 DEJMPS-style circuit (default)
    qr = QuantumRegister(4, 'q')
    cr = ClassicalRegister(3, 'c')
    qc = QuantumCircuit(qr, cr)

    # Bilateral CNOT
    qc.cx(qr[1], qr[0])  # Alice: q1 -> q0
    qc.cx(qr[2], qr[3])  # Bob:   q2 -> q3

    # Measure the discard pair
    qc.measure(qr[0], cr[0])
    qc.measure(qr[3], cr[1])

    # Compute parity flag into c2
    qc.reset(qr[0])
    with qc.if_test((cr[0], 1)):
        qc.x(qr[0])
    with qc.if_test((cr[1], 1)):
        qc.x(qr[0])

    qc.measure(qr[0], cr[2])

    return qc

def create_4to1_distillation_circuit():
    # 4-to-1 recurrence-style circuit (improved)
    qr = QuantumRegister(8, 'q')
    cr = ClassicalRegister(4, 'c')
    qc = QuantumCircuit(qr, cr)

    # Pairing: (0,7), (1,6), (2,5), (3,4)
    # Step 1: Bilateral CNOTs for all pairs except the final pair (3,4)
    qc.cx(qr[1], qr[0])  # Alice: q1 -> q0
    qc.cx(qr[2], qr[3])  # Alice: q2 -> q3
    qc.cx(qr[5], qr[4])  # Bob:   q5 -> q4
    qc.cx(qr[6], qr[7])  # Bob:   q6 -> q7

    # Step 2: Measure and reset discard pairs (0,7), (1,6), (2,5)
    qc.measure(qr[0], cr[0])
    qc.measure(qr[7], cr[1])
    qc.measure(qr[1], cr[2])
    qc.measure(qr[6], cr[3])

    # Step 3: Parity check and correction (example logic, can be improved)
    # Reset and correct the final pair (3,4) based on measured bits
    qc.reset(qr[3])
    qc.reset(qr[4])
    # Example: flip if any measured bit is 1 (parity check)
    for i in range(4):
        with qc.if_test((cr[i], 1)):
            qc.x(qr[3])
            qc.x(qr[4])

    # Final measurement (optional, for post-selection)
    # qc.measure(qr[3], cr[0])
    # qc.measure(qr[4], cr[1])

    return qc

# Example usage:
circuit = create_distillation_circuit()
circuit4 = create_4to1_distillation_circuit()
circuit.draw(output='text')
circuit4.draw(output='text')

┌───┐┌─┐                                                               »
q_0: ┤ X ├┤M├───────────────────────────────────────────────────────────────»
     └─┬─┘└╥┘     ┌─┐                                                       »
q_1: ──■───╫──────┤M├───────────────────────────────────────────────────────»
           ║      └╥┘                                                       »
q_2: ──■───╫───────╫────────────────────────────────────────────────────────»
     ┌─┴─┐ ║       ║      ┌──────  ┌───┐ ───────┐   ┌──────  ┌───┐ ───────┐ »
q_3: ┤ X ├─╫──|0>──╫──────┤       ─┤ X ├        ├───┤       ─┤ X ├        ├─»
     ├───┤ ║       ║      │ If-0   ├───┤  End-0 │   │ If-0   ├───┤  End-0 │ »
q_4: ┤ X ├─╫──|0>──╫──────┤       ─┤ X ├        ├───┤       ─┤ X ├        ├─»
     └─┬─┘ ║       ║      └──╥───  └───┘ ───────┘   └──╥───  └───┘ ───────┘ »
q_5: ──■───╫───────╫─────────╫─────────────────────────╫────────────────────»
           ║       ║ ┌─┐     ║                         ║                    »
q_6: ──■───╫───────╫─┤M├─────╫─────────────────────────╫────────────────────»
     ┌─┴─┐ ║  ┌─┐  ║ └╥┘     ║                         ║                    »
q_7: ┤ X ├─╫──┤M├──╫──╫──────╫─────────────────────────╫────────────────────»
     └───┘ ║  └╥┘  ║  ║ ┌────╨────┐               ┌────╨────┐               »
c: 4/══════╩═══╩═══╩══╩═╡ c_0=0x1 ╞═══════════════╡ c_1=0x1 ╞═══════════════»
           0   1   2  3 └─────────┘               └─────────┘               »
«                                                         
«q_0: ────────────────────────────────────────────────────
«                                                         
«q_1: ────────────────────────────────────────────────────
«                                                         
«q_2: ────────────────────────────────────────────────────
«       ┌──────  ┌───┐ ───────┐   ┌──────  ┌───┐ ───────┐ 
«q_3: ──┤       ─┤ X ├        ├───┤       ─┤ X ├        ├─
«       │ If-0   ├───┤  End-0 │   │ If-0   ├───┤  End-0 │ 
«q_4: ──┤       ─┤ X ├        ├───┤       ─┤ X ├        ├─
«       └──╥───  └───┘ ───────┘   └──╥───  └───┘ ───────┘ 
«q_5: ─────╫─────────────────────────╫────────────────────
«          ║                         ║                    
«q_6: ─────╫─────────────────────────╫────────────────────
«          ║                         ║                    
«q_7: ─────╫─────────────────────────╫────────────────────
«     ┌────╨────┐               ┌────╨────┐               
«c: 4/╡ c_2=0x1 ╞═══════════════╡ c_3=0x1 ╞═══════════════
«     └─────────┘               └─────────┘

## Step 5: Claim an Edge

In [826]:
from collections import deque, defaultdict
import requests
GRAPH_URL = "https://demo-entanglement-distillation-qfhvrahfcq-uc.a.run.app/v1/graph"
response = requests.get(GRAPH_URL)
graph_data = response.json()

# Defensive: handle API response with 'data' key
if "data" in graph_data:
    graph_data = graph_data["data"]

nodes = {n['node_id']: n for n in graph_data['nodes']}
adj = defaultdict(list)
for edge in graph_data["edges"]:
    n1, n2 = edge["edge_id"]
    adj[n1].append(n2)
    adj[n2].append(n1)

# Find all nodes with a bonus, sorted by bonus descending
bonus_nodes = sorted(
    [n for n in nodes if nodes[n].get("bonus_bell_pairs", 0) > 0],
    key=lambda n: -nodes[n]["bonus_bell_pairs"]
)

owned = set(client.get_status().get("owned_nodes", []))
claimable = client.get_claimable_edges()

def min_distance_to_bonus(start, adj, bonus_nodes):
    """BFS from start node to nearest bonus node."""
    visited = set()
    queue = deque([(start, 0)])
    while queue:
        node, dist = queue.popleft()
        if node in bonus_nodes:
            return dist
        for neighbor in adj[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, dist + 1))
    return float('inf')

def edge_priority(edge):
    n1, n2 = edge["edge_id"]
    # Always sort by the node we do NOT own
    if n1 in owned and n2 not in owned:
        target_node = n2
    elif n2 in owned and n1 not in owned:
        target_node = n1
    else:
        # If both are unowned or both owned, pick the one with higher bonus
        if nodes[n1].get('bonus_bell_pairs', 0) >= nodes[n2].get('bonus_bell_pairs', 0):
            target_node = n1
        else:
            target_node = n2
    dist = min_distance_to_bonus(target_node, adj, bonus_nodes)
    bonus = nodes[target_node].get('bonus_bell_pairs', 0)
    # Lower distance is better, then more bonus, then easier, then lower threshold
    return (dist, -bonus, -edge.get('bonus_bell_pairs', 0), -edge['difficulty_rating'], edge['base_threshold'])

# Remove duplicate targets: only keep the best edge for each target node
seen_targets = set()
unique_claimable_sorted = []
for edge in sorted(claimable, key=edge_priority):
    n1, n2 = edge["edge_id"]
    if n1 in owned and n2 not in owned:
        target_node = n2
    elif n2 in owned and n1 not in owned:
        target_node = n1
    else:
        # If both are unowned or both owned, pick the one with higher bonus
        if nodes[n1].get('bonus_bell_pairs', 0) >= nodes[n2].get('bonus_bell_pairs', 0):
            target_node = n1
        else:
            target_node = n2
    if target_node in seen_targets:
        continue
    seen_targets.add(target_node)
    unique_claimable_sorted.append(edge)

print(f"Claimable edges ({len(unique_claimable_sorted)} unique targets):")
for idx, edge in enumerate(unique_claimable_sorted):
    n1, n2 = edge['edge_id']
    # Show the bonus for the node we do NOT own
    if n1 in owned and n2 not in owned:
        bonus_target = nodes[n2].get('bonus_bell_pairs', 0)
        target_label = n2
    elif n2 in owned and n1 not in owned:
        bonus_target = nodes[n1].get('bonus_bell_pairs', 0)
        target_label = n1
    else:
        # If both are unowned or both owned, show both
        bonus_target = f"{nodes[n1].get('bonus_bell_pairs', 0)}/{nodes[n2].get('bonus_bell_pairs', 0)}"
        target_label = f"{n1}/{n2}"
    print(
        f"{idx:2d}: {edge['edge_id']} - threshold: {edge['base_threshold']:.3f}, ",
        f"difficulty: {edge['difficulty_rating']}, ",
        f"bonus (target {target_label}): {bonus_target} ",
        f"edge bonus: {edge.get('bonus_bell_pairs', 0)}"
    )

Claimable edges (54 unique targets):
 0: ['Hanover, NH', 'Worcester, MA'] - threshold: 0.900,  difficulty: 2,  bonus (target Hanover, NH): 3  edge bonus: 0
 1: ['Boston, MA', 'Dublin, Ireland'] - threshold: 0.920,  difficulty: 4,  bonus (target Dublin, Ireland): 2  edge bonus: 0
 2: ['Buenos Aires, Argentina', 'Santiago, Chile'] - threshold: 0.920,  difficulty: 4,  bonus (target Santiago, Chile): 2  edge bonus: 0
 3: ['Charlottesville, VA', 'College Park, MD'] - threshold: 0.900,  difficulty: 2,  bonus (target Charlottesville, VA): 2  edge bonus: 0
 4: ['Raleigh, NC', 'Richmond, VA'] - threshold: 0.900,  difficulty: 2,  bonus (target Raleigh, NC): 2  edge bonus: 0
 5: ['Cleveland, OH', 'Pittsburgh, PA'] - threshold: 0.900,  difficulty: 2,  bonus (target Cleveland, OH): 2  edge bonus: 0
 6: ['Columbus, OH', 'Pittsburgh, PA'] - threshold: 0.900,  difficulty: 2,  bonus (target Columbus, OH): 2  edge bonus: 0
 7: ['Birmingham, UK', 'London, UK'] - threshold: 0.900,  difficulty: 2,  bonus (

In [827]:
import time
if claimable:
    goal = 10  # Claim the first 10 edges in the sorted list
    indy = 0
    attempt = 0
    claimable_sorted = sorted(claimable, key=edge_priority)
    while attempt < 1000 and indy < goal:
        if indy >= len(claimable_sorted):
            print("No more claimable edges available.")
            break
        target = claimable_sorted[indy]  # Next highest-priority edge
        edge_id = tuple(target['edge_id'])
        success = False
        while attempt < 1000 and not success:
            circuit = create_distillation_circuit()
            num_bell_pairs = 2
            flag_bit = 2  # Classical bit for post-selection (keep when flag=0)
            attempt += 1
            print(f"Attempt {attempt}: Claiming {edge_id} (threshold: {target['base_threshold']:.3f})...")
            try:
                result = client.claim_edge(edge_id, circuit, flag_bit, num_bell_pairs)
                if result.get("ok"):
                    data = result["data"]
                    print(f"Success: {data.get('success')}")
                    print(f"Fidelity: {data.get('fidelity', 0):.4f} (threshold: {data.get('threshold', 0):.4f})")
                    print(f"Success probability: {data.get('success_probability', 0):.4f}")
                    if data.get('success'):
                        print("✅ Edge successfully claimed!")
                        success = True
                        # Refresh claimable and sorted list after each claim
                        claimable = client.get_claimable_edges()
                        if not claimable:
                            print("No more claimable edges available.")
                            break
                        claimable_sorted = sorted(claimable, key=edge_priority)
                        break
                    else:
                        print("❌ Attempt failed (free). Retrying...")
                else:
                    print(f"Error: {result.get('error', {}).get('message')}")
                    break
            except Exception as e:
                import traceback
                print("Exception occurred while claiming edge:", e)
                traceback.print_exc()
                # Try to get more info from the HTTPError if possible
                if hasattr(e, 'response') and e.response is not None:
                    try:
                        print('Server response:', e.response.text)
                    except Exception as e3:
                        print('Could not print server response:', e3)
                # Try fallback: use 2-to-1 circuit if 4-to-1 fails
                print("Trying fallback: 2-to-1 circuit with 2 bell pairs...")
                try:
                    circuit2 = create_distillation_circuit()
                    num_bell_pairs2 = 2
                    result2 = client.claim_edge(edge_id, circuit2, flag_bit, num_bell_pairs2)
                    if result2.get("ok"):
                        data2 = result2["data"]
                        print(f"Success: {data2.get('success')}")
                        print(f"Fidelity: {data2.get('fidelity', 0):.4f} (threshold: {data2.get('threshold', 0):.4f})")
                        print(f"Success probability: {data2.get('success_probability', 0):.4f}")
                        if data2.get('success'):
                            print("✅ Edge successfully claimed with 2-to-1 circuit!")
                            success = True
                            # Refresh claimable and sorted list after each claim
                            claimable = client.get_claimable_edges()
                            if not claimable:
                                print("No more claimable edges available.")
                                break
                            claimable_sorted = sorted(claimable, key=edge_priority)
                            break
                        else:
                            print("❌ Attempt failed (free) with 2-to-1 circuit. Retrying...")
                    else:
                        print(f"Error: {result2.get('error', {}).get('message')}")
                        break
                except Exception as e2:
                    print("Fallback also failed:", e2)
                    if hasattr(e2, 'response') and e2.response is not None:
                        try:
                            print('Server response:', e2.response.text)
                        except Exception as e3:
                            print('Could not print server response:', e3)
                    break
            time.sleep(1)  # Avoid spamming the server
        if success:
            indy += 1

Attempt 1: Claiming ('Hanover, NH', 'Worcester, MA') (threshold: 0.900)...
Success: False
Fidelity: 0.8095 (threshold: 0.9000)
Success probability: 0.8400
❌ Attempt failed (free). Retrying...
Attempt 2: Claiming ('Hanover, NH', 'Worcester, MA') (threshold: 0.900)...
Success: False
Fidelity: 0.2000 (threshold: 0.9000)
Success probability: 0.2000
❌ Attempt failed (free). Retrying...
Attempt 3: Claiming ('Hanover, NH', 'Worcester, MA') (threshold: 0.900)...
Success: False
Fidelity: 0.8095 (threshold: 0.9000)
Success probability: 0.8400
❌ Attempt failed (free). Retrying...
Attempt 4: Claiming ('Hanover, NH', 'Worcester, MA') (threshold: 0.900)...
Success: False
Fidelity: 0.0000 (threshold: 0.9000)
Success probability: 0.3200
❌ Attempt failed (free). Retrying...
Attempt 5: Claiming ('Hanover, NH', 'Worcester, MA') (threshold: 0.900)...
Success: False
Fidelity: 0.8095 (threshold: 0.9000)
Success probability: 0.8400
❌ Attempt failed (free). Retrying...
Attempt 6: Claiming ('Hanover, NH', 'Wor

Traceback (most recent call last):
  File "C:\Users\gebha\AppData\Local\Temp\ipykernel_35936\1126839976.py", line 21, in <module>
    result = client.claim_edge(edge_id, circuit, flag_bit, num_bell_pairs)
  File "c:\Users\gebha\Documents\Code\StudentClones\2026-IonQ\client.py", line 108, in claim_edge
    return self._post("/v1/claim_edge", payload)
           ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gebha\Documents\Code\StudentClones\2026-IonQ\client.py", line 35, in _post
    r.raise_for_status()
    ~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\gebha\AppData\Roaming\Python\Python313\site-packages\requests\models.py", line 1024, in raise_for_status
requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://demo-entanglement-distillation-qfhvrahfcq-uc.a.run.app/v1/claim_edge


Success: True
Fidelity: 1.0000 (threshold: 0.9000)
Success probability: 0.0400
✅ Edge successfully claimed with 2-to-1 circuit!
Attempt 35: Claiming ('Raleigh, NC', 'Richmond, VA') (threshold: 0.900)...
Success: False
Fidelity: 0.8000 (threshold: 0.9000)
Success probability: 0.8000
❌ Attempt failed (free). Retrying...
Attempt 36: Claiming ('Raleigh, NC', 'Richmond, VA') (threshold: 0.900)...
Success: False
Fidelity: 0.8000 (threshold: 0.9000)
Success probability: 0.8000
❌ Attempt failed (free). Retrying...
Attempt 37: Claiming ('Raleigh, NC', 'Richmond, VA') (threshold: 0.900)...
Success: True
Fidelity: 1.0000 (threshold: 0.9000)
Success probability: 0.6800
✅ Edge successfully claimed!
Attempt 38: Claiming ('Cleveland, OH', 'Pittsburgh, PA') (threshold: 0.900)...
Success: False
Fidelity: 0.6800 (threshold: 0.9000)
Success probability: 1.0000
❌ Attempt failed (free). Retrying...
Attempt 39: Claiming ('Cleveland, OH', 'Pittsburgh, PA') (threshold: 0.900)...
Success: False
Fidelity: 0.666

Traceback (most recent call last):
  File "C:\Users\gebha\AppData\Local\Temp\ipykernel_35936\1126839976.py", line 21, in <module>
    result = client.claim_edge(edge_id, circuit, flag_bit, num_bell_pairs)
  File "c:\Users\gebha\Documents\Code\StudentClones\2026-IonQ\client.py", line 108, in claim_edge
    return self._post("/v1/claim_edge", payload)
           ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gebha\Documents\Code\StudentClones\2026-IonQ\client.py", line 35, in _post
    r.raise_for_status()
    ~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\gebha\AppData\Roaming\Python\Python313\site-packages\requests\models.py", line 1024, in raise_for_status
requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://demo-entanglement-distillation-qfhvrahfcq-uc.a.run.app/v1/claim_edge


Fallback also failed: 400 Client Error: Bad Request for url: https://demo-entanglement-distillation-qfhvrahfcq-uc.a.run.app/v1/claim_edge
Server response: {"ok":false,"error":{"code":"EDGE_NOT_ADJACENT","message":"Edge must connect owned->unowned node"}}
Attempt 48: Claiming ('Cleveland, OH', 'Pittsburgh, PA') (threshold: 0.900)...
Exception occurred while claiming edge: 400 Client Error: Bad Request for url: https://demo-entanglement-distillation-qfhvrahfcq-uc.a.run.app/v1/claim_edge
Server response: {"ok":false,"error":{"code":"EDGE_NOT_ADJACENT","message":"Edge must connect owned->unowned node"}}
Trying fallback: 2-to-1 circuit with 2 bell pairs...


...

Traceback (most recent call last):
  File "C:\Users\gebha\AppData\Local\Temp\ipykernel_35936\1126839976.py", line 21, in <module>
    result = client.claim_edge(edge_id, circuit, flag_bit, num_bell_pairs)
  File "c:\Users\gebha\Documents\Code\StudentClones\2026-IonQ\client.py", line 108, in claim_edge
    return self._post("/v1/claim_edge", payload)
           ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gebha\Documents\Code\StudentClones\2026-IonQ\client.py", line 35, in _post
    r.raise_for_status()
    ~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\gebha\AppData\Roaming\Python\Python313\site-packages\requests\models.py", line 1024, in raise_for_status
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://demo-entanglement-distillation-qfhvrahfcq-uc.a.run.app/v1/claim_edge


Fallback also failed: 403 Client Error: Forbidden for url: https://demo-entanglement-distillation-qfhvrahfcq-uc.a.run.app/v1/claim_edge
Server response: {"ok":false,"error":{"code":"PLAYER_INACTIVE","message":"Player eliminated (budget exhausted)"}}
Attempt 109: Claiming ('Cleveland, OH', 'Pittsburgh, PA') (threshold: 0.900)...
Exception occurred while claiming edge: 403 Client Error: Forbidden for url: https://demo-entanglement-distillation-qfhvrahfcq-uc.a.run.app/v1/claim_edge
Server response: {"ok":false,"error":{"code":"PLAYER_INACTIVE","message":"Player eliminated (budget exhausted)"}}
Trying fallback: 2-to-1 circuit with 2 bell pairs...


Traceback (most recent call last):
  File "C:\Users\gebha\AppData\Local\Temp\ipykernel_35936\1126839976.py", line 21, in <module>
    result = client.claim_edge(edge_id, circuit, flag_bit, num_bell_pairs)
  File "c:\Users\gebha\Documents\Code\StudentClones\2026-IonQ\client.py", line 108, in claim_edge
    return self._post("/v1/claim_edge", payload)
           ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gebha\Documents\Code\StudentClones\2026-IonQ\client.py", line 35, in _post
    r.raise_for_status()
    ~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\gebha\AppData\Roaming\Python\Python313\site-packages\requests\models.py", line 1024, in raise_for_status
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://demo-entanglement-distillation-qfhvrahfcq-uc.a.run.app/v1/claim_edge


Fallback also failed: 403 Client Error: Forbidden for url: https://demo-entanglement-distillation-qfhvrahfcq-uc.a.run.app/v1/claim_edge
Server response: {"ok":false,"error":{"code":"PLAYER_INACTIVE","message":"Player eliminated (budget exhausted)"}}
Attempt 110: Claiming ('Cleveland, OH', 'Pittsburgh, PA') (threshold: 0.900)...


KeyboardInterrupt: 

## Step 6: Check Progress

In [807]:
client.print_status()

Player: diyamagnetism (Diya Naik)
Score: 66 | Budget: 28 bell pairs
Active: Yes
Starting node: Brussels, Belgium
Owned: 26 nodes, 38 edges
Claimable edges: 67
  - ['Miami, FL', 'Nassau, Bahamas']: threshold=0.90, difficulty=2
  - ['Havana, Cuba', 'Miami, FL']: threshold=0.90, difficulty=3
  - ['Jacksonville, FL', 'Orlando, FL']: threshold=0.90, difficulty=2
  ... and 64 more


In [17]:
# View leaderboard
leaderboard = client.get_leaderboard()["leaderboard"]
print("Leaderboard:")
for i, p in enumerate(leaderboard[:10]):
    print(f"{i+1}. {p.get('player_id', 'Unknown'):20} Score: {p.get('score', 0)}")

Leaderboard:
1. ahkatlio_76f0869e    Score: 7
2. lasse                Score: 0
3. your_unique_id       Score: 0
4. FEY                  Score: 0
5. 010104               Score: 0
6. wwwillott            Score: 0
7. splch                Score: 0
8. JasonAmosTroll       Score: 0
9. 1                    Score: 0
10. savagejflanc         Score: 0


## Tips

- **Failed attempts are free** - only successful claims cost bell pairs
- **More bell pairs** can improve fidelity but cost more budget
- **Vertex rewards** are competitive - top players by claim strength earn rewards
- **Budget management** is key - if budget reaches 0, you're eliminated

Good luck!

In [106]:
result = client.restart()
print(result)

{'ok': True, 'data': {'success': True, 'budget': 75, 'is_active': True, 'starting_candidates': [{'node_id': 'Vilnius, Lithuania', 'utility_qubits': 2, 'bonus_bell_pairs': 1, 'capacity': 5, 'latitude': 54.6872, 'longitude': 25.2797}, {'node_id': 'Phnom Penh, Cambodia', 'utility_qubits': 3, 'bonus_bell_pairs': 0, 'capacity': 5, 'latitude': 11.5564, 'longitude': 104.9282}, {'node_id': 'Toulouse, France', 'utility_qubits': 3, 'bonus_bell_pairs': 2, 'capacity': 7, 'latitude': 43.6047, 'longitude': 1.4442}, {'node_id': 'Pune, India', 'utility_qubits': 4, 'bonus_bell_pairs': 2, 'capacity': 7, 'latitude': 18.5204, 'longitude': 73.8567}]}}


In [162]:
claimable = client.get_claimable_edges()  # This is already a list

print("Claimable edges:")
for e in claimable:
    print(
        e["edge_id"],
        "threshold =", e["base_threshold"],
        "difficulty =", e["difficulty_rating"]
    )

Claimable edges:
['Barcelona, Spain', 'Toulouse, France'] threshold = 0.9 difficulty = 2
['Montpellier, France', 'Toulouse, France'] threshold = 0.9 difficulty = 2
['Bordeaux, France', 'Toulouse, France'] threshold = 0.9 difficulty = 2
['Marseille, France', 'Toulouse, France'] threshold = 0.9 difficulty = 2
['Bilbao, Spain', 'Toulouse, France'] threshold = 0.9 difficulty = 3


In [164]:
def probe_edge(edge):
    edge_id = tuple(edge["edge_id"])
    circuit = create_distillation_circuit()

    result = client.claim_edge(
        edge_id,
        circuit,
        flag_bit=2,
        num_bell_pairs=2
    )

    if result.get("ok"):
        data = result["data"]
        return data["fidelity"], data["success_probability"]

    return None, None

# Select a starting node if not already selected
status = client.get_status()
if not status.get("starting_node"):
    candidates = status.get("starting_candidates", [])
    if candidates:
        node_id = candidates[0]["node_id"]  # Or let user pick
        result = client.select_starting_node(node_id)
        print("Selected starting node:", node_id)
    else:
        print("No starting candidates available.")

# Now you can safely probe edges
claimable = client.get_claimable_edges()
print("\n=== Probing edges ===")
for edge in claimable:
    fid, ps = probe_edge(edge)
    print(edge["edge_id"], "F =", fid, "P =", ps)



=== Probing edges ===
['Barcelona, Spain', 'Toulouse, France'] F = 0.8095238095238095 P = 0.8400000000000001
['Montpellier, France', 'Toulouse, France'] F = 1.0295220005548027e-34 P = 0.16000000000000003
['Bordeaux, France', 'Toulouse, France'] F = 0.8 P = 0.8
['Marseille, France', 'Toulouse, France'] F = 1.0 P = 0.04000000000000001
['Bilbao, Spain', 'Toulouse, France'] F = 0.0 P = 0.25000000000000006


In [167]:
edge_id = ("Barcelona, Spain", "Toulouse, France")

result = client.claim_edge(
    edge_id,
    create_distillation_circuit(),
    flag_bit=2,
    num_bell_pairs=2
)

print(result)


{'ok': True, 'data': {'success': False, 'is_valid': False, 'fidelity': 0.1111111111111111, 'success_probability': 0.3600000000000001, 'threshold': 0.9, 'remaining_budget': 76, 'is_active': True, 'score': 10, 'reward_changes': []}}


In [170]:
import time

# -----------------------------
# Automated Distillation Claimer
# -----------------------------

def probe_edge(edge, num_bell_pairs=2):
    """Probe an edge for fidelity + success probability (free attempt)."""
    edge_id = tuple(edge["edge_id"])
    circuit = create_distillation_circuit()

    result = client.claim_edge(edge_id, circuit, flag_bit=2, num_bell_pairs=num_bell_pairs)

    if result.get("ok"):
        data = result["data"]
        return data["fidelity"], data["success_probability"], data["threshold"]
    return None, None, None


def auto_claim_edges(max_claims=5):
    """
    Loop:
    1. Probe all edges
    2. Pick best candidate
    3. Claim it
    4. Repeat
    """

    print("\n=== Starting Auto-Claim Mode ===")

    confirm = input("Run automated claiming loop? (y/n): ")
    if confirm.lower() != "y":
        print("Cancelled.")
        return

    claims_done = 0

    while claims_done < max_claims:

        # Refresh claimable edges
        resp = client.get_claimable_edges()
        claimable = client.get_claimable_edges()

        if not claimable:
            print("No claimable edges available.")
            break

        print(f"\n--- Probing {len(claimable)} edges ---")

        scored_edges = []

        # Probe each edge
        for edge in claimable:
            edge_id = tuple(edge["edge_id"])
            fid, psucc, thresh = probe_edge(edge)

            if fid is None:
                continue

            score = fid * psucc

            print(
                f"Edge {edge_id} | F={fid:.3f}, P={psucc:.3f}, "
                f"Score={score:.3f}, Threshold={thresh:.3f}"
            )

            scored_edges.append((score, fid, psucc, edge))

        if not scored_edges:
            print("No valid probe results.")
            break

        # Pick best edge by score
        scored_edges.sort(reverse=True, key=lambda x: x[0])
        best_score, best_fid, best_psucc, target = scored_edges[0]

        edge_id = tuple(target["edge_id"])
        threshold = target["base_threshold"]

        print("\n==============================")
        print(f"Best candidate: {edge_id}")
        print(f" Probe Fidelity: {best_fid:.4f}")
        print(f" Success Prob:   {best_psucc:.4f}")
        print(f" Threshold:      {threshold:.4f}")
        print("==============================")

        # If probe fidelity is below threshold, skip
        if best_fid < threshold:
            print("❌ Best edge still cannot beat threshold. Stopping loop.")
            break

        # -----------------------------
        # Claim an edge (your format)
        # -----------------------------
        circuit = create_distillation_circuit()
        num_bell_pairs = 2
        flag_bit = 2

        print(f"\nClaiming {edge_id} (threshold: {threshold:.3f})...")

        result = client.claim_edge(edge_id, circuit, flag_bit, num_bell_pairs)

        if result.get("ok"):
            data = result["data"]

            print(f"Success: {data.get('success')}")
            print(f"Fidelity: {data.get('fidelity', 0):.4f} "
                  f"(threshold: {data.get('threshold', 0):.4f})")
            print(f"Success probability: {data.get('success_probability', 0):.4f}")

            if data.get("success"):
                claims_done += 1
                print(f"✅ Claims completed: {claims_done}/{max_claims}")
            else:
                print("❌ Claim attempt failed (free). Moving on.")

        else:
            print(f"Error: {result.get('error', {}).get('message')}")

        # Small delay so you don't spam server
        time.sleep(1)

    print("\n=== Auto-Claim Loop Finished ===")


In [171]:
auto_claim_edges()


=== Starting Auto-Claim Mode ===

--- Probing 12 edges ---
Edge ('Marseille, France', 'Montpellier, France') | F=0.970, P=0.745, Score=0.723, Threshold=0.900
Edge ('Marseille, France', 'Nice, France') | F=0.970, P=0.745, Score=0.723, Threshold=0.900
Edge ('Grenoble, France', 'Marseille, France') | F=0.000, P=0.160, Score=0.000, Threshold=0.900
Edge ('Marseille, France', 'Turin, Italy') | F=1.000, P=0.040, Score=0.040, Threshold=0.900
Edge ('Barcelona, Spain', 'Montpellier, France') | F=0.800, P=0.800, Score=0.640, Threshold=0.900


HTTPError: 400 Client Error: Bad Request for url: https://demo-entanglement-distillation-qfhvrahfcq-uc.a.run.app/v1/claim_edge